# How to insert, update, and delete multiple records with Dapper.Contrib.Extensions

Dapper.Contrib.Extensions is a library that extends the <code>IDbConnection</code> interface with methods for CRUD operations on your entities. You can use these methods to insert, update, and delete data from a database using C# and Dapper. To use Dapper.Contrib, you need to add attributes like <code>[Table]</code> and <code>[Key]</code> to your entity class, Product. These attributes indicate the table name and the primary key. Then you can pass the Product object to the appropriate method, and Dapper.Contrib will create the SQL query for you. For example:

In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:System.ComponentModel"
#r "nuget:Dapper"
#r "nuget:Dapper.Contrib"

using Microsoft.Data.SqlClient;
using System;
using System.Collections.Generic;
using System.Data;
using Dapper;
using Dapper.Contrib.Extensions;

Installed Packages Dapper, 2.1.24 Dapper.Contrib, 2.0.78 Microsoft.Data.SqlClient, 5.1.2 System.ComponentModel, 4.3.0

You need to decorate your entity class with the [Table] attribute and specify the name of the table in the database. For example, if you have a Product class that corresponds to the Production.Product table in the AdventureWorks2022 database, you can write:

In [3]:
[Table("Production.Product")]
public class Product
{    
    [Key]
    public int ProductID { get; set; }
    public string Name { get; set; }
    public string ProductNumber { get; set; }
    public string Color { get; set; }
    public decimal StandardCost { get; set; }
    public decimal ListPrice { get; set; }
    public int SafetyStockLevel { get; set; }
    public int ReorderPoint { get; set; }
    public int DaysToManufacture { get; set; }
    public DateTime SellStartDate { get; set; }
}

To insert new records into the Production.Product table:

In [4]:
var rand = new Random();
var productsToInsert = new List<Product>()
{
    new Product() { Name = "Bike Helmet-" + rand.Next(1000, 9999), ProductNumber = "BK-H01-" + rand.Next(1000, 9999), Color = "Red", StandardCost = 15.50m, ListPrice = 34.99m, 
                    SafetyStockLevel = 999, ReorderPoint = 777, DaysToManufacture = 1, SellStartDate = new DateTime(2000, 1, 2) },    
    new Product() { Name = "Gloves-" + rand.Next(1000, 9999), ProductNumber = "GL-H01-" + rand.Next(1000, 9999), Color = "Black", StandardCost = 5.00m, ListPrice = 9.99m, 
                    SafetyStockLevel = 999, ReorderPoint = 777, DaysToManufacture = 1, SellStartDate = new DateTime(2000, 1, 2) }
};

List<Product> newPoducts;
long count;
using (var connection = new SqlConnection(connectionString))
{
    connection.Open();
    count = connection.Insert(productsToInsert);

    // Query the Product table and display the results   
    newPoducts = connection.Query<Product>($"SELECT top {count} * FROM Production.Product order by ProductID desc").ToList();

    foreach (var product in newPoducts)
    {
        Console.WriteLine($"{product.ProductID}, {product.Name}, {product.ProductNumber}, {product.Color}, {product.StandardCost}, {product.ListPrice}");
    }
}

1170, Gloves-7640, GL-H01-4875, Black, 5.0000, 9.9900
1169, Bike Helmet-1205, BK-H01-6413, Red, 15.5000, 34.9900


To update the above records:

In [5]:
var productsToUpdate = new List<Product>()
{
    new Product() { ProductID = newPoducts[0].ProductID, Name = "Adjustable Race-" + rand.Next(1000, 9999), ProductNumber = "AR-" + rand.Next(1000, 9999), 
                    Color = "Silver", StandardCost = 300.00m, ListPrice = 400.00m, SafetyStockLevel = 555, ReorderPoint = 747, DaysToManufacture = 0, 
                    SellStartDate = DateTime.Now },   
    new Product() { ProductID = newPoducts[1].ProductID, Name = "BB Ball Bearing-" + rand.Next(1000, 9999), ProductNumber = "BE-" + rand.Next(1000, 9999), 
                    Color = "Silver", StandardCost = 0.75m, ListPrice = 1.49m, SafetyStockLevel = 555, ReorderPoint = 747, DaysToManufacture = 0, 
                    SellStartDate = DateTime.Now }
};

using (var connection = new SqlConnection(connectionString))
{
    connection.Open();
    connection.Update(productsToUpdate);

    // Query the Product table and display the results   
    foreach(Product product in productsToUpdate)                    
    {
        Product p = connection.Get<Product>(product.ProductID);
        Console.WriteLine($"{p.ProductID}, {p.Name}, {p.ProductNumber}, {p.Color}, {p.StandardCost}, {p.ListPrice}, {p.SafetyStockLevel}, {p.DaysToManufacture}, {p.SellStartDate}");
    } 
}

1170, Adjustable Race-5633, AR-2159, Silver, 300.0000, 400.0000, 555, 0, 12/23/2023 6:01:07 PM
1169, BB Ball Bearing-5069, BE-9471, Silver, 0.7500, 1.4900, 555, 0, 12/23/2023 6:01:07 PM


To delete the above records:

In [6]:
using (var connection = new SqlConnection(connectionString))
{
    connection.Open();
    connection.Delete(newPoducts);

    // Query the Product table and display the results, which have been deleted already.    
    var sql = $@"SELECT ProductID, Name, ProductNumber, Color, StandardCost, ListPrice, SafetyStockLevel, DaysToManufacture, SellStartDate
                        FROM Production.Product
                        WHERE ProductID = @ProductID";

   foreach(Product product in productsToUpdate)                    
    {
        Product p = connection.Get<Product>(product.ProductID);        
        Console.WriteLine(p == null);
    } 
}


True
True
